In [1]:
import pandas as pd
import requests
import sqlite3

from collections import defaultdict
from pathlib import Path

In [17]:
df_street = pd.read_csv('2023-06-10-requerimento-ruas.csv')

df_street

,requerimento,ano,numero,ementa,autores,id,numero_protocolo,texto_original,DOC,OCCUPATION,PERSON,REGION,ORG,STREET,MISC,LOC,DATE
0,Requerimento de Vereador(a) nº 5 de 2021,2021,5,Solicita ao Executivo pavimentação asfáltica n...,14,2671,63.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,"('Jardim Monte das Oliveiras', 'Bairro Santo A...","('Executivo',)",Rua William Vilas Boas,NaN,NaN,NaN
1,Requerimento de Vereador(a) nº 7 de 2021,2021,7,Solicita ao Executivo a implantação de faixa e...,14,2673,65.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,"('Vila Santa Terezinha',)","('Executivo',)",Rua William Vilas Boas,NaN,NaN,NaN
2,Requerimento de Vereador(a) nº 14 de 2021,2021,14,Solicita ao Executivo uma faixa elevada na Rua...,20,2683,74.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,NaN,"('Executivo',)",Rua 13 de Maio,NaN,NaN,NaN
3,Requerimento de Vereador(a) nº 14 de 2021,2021,14,Solicita ao Executivo uma faixa elevada na Rua...,20,2683,74.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,NaN,"('Executivo',)",Rua Dep.,NaN,NaN,NaN
4,Requerimento de Vereador(a) nº 14 de 2021,2021,14,Solicita ao Executivo uma faixa elevada na Rua...,20,2683,74.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,NaN,"('Executivo',)",Benedito Lúcio Machado,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,Requerimento de Vereador(a) nº 281 de 2023,2023,281,Solicita ao Departamento Municipal de Energia ...,20,4603,856.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,NaN,"('Departamento Municipal de Energia Elétrica',)",Rua dos Estudantes,NaN,NaN,NaN
614,Requerimento de Vereador(a) nº 288 de 2023,2023,288,Solicita à Secretaria Municipal de Serviços e ...,13,4613,879.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,"('Requerimento nº 528/2020',)",NaN,NaN,"('Jardim Santa Terezinha',)",('Secretaria Municipal de Serviços e Obras Púb...,Rua Willian Villas Boas,NaN,NaN,NaN
615,Requerimento de Vereador(a) nº 288 de 2023,2023,288,Solicita à Secretaria Municipal de Serviços e ...,13,4613,879.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,"('Requerimento nº 528/2020',)",NaN,NaN,"('Jardim Santa Terezinha',)",('Secretaria Municipal de Serviços e Obras Púb...,Rua Dirceu Galvão,NaN,NaN,NaN
616,Requerimento de Vereador(a) nº 290 de 2023,2023,290,Solicita ao Departamento Municipal de Trânsito...,20,4620,920.0,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,"('Jardim Eldorado',)","('Departamento Municipal de Trânsito',)",Rua Lucélia Ribeiro Campos e Silva,NaN,NaN,NaN


In [18]:
columns = [
    'requerimento',
    'numero_protocolo',
    'DOC',
]

df_street = df_street.drop(columns=columns)

df_street.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ano             618 non-null    int64 
 1   numero          618 non-null    int64 
 2   ementa          618 non-null    object
 3   autores         618 non-null    object
 4   id              618 non-null    int64 
 5   texto_original  618 non-null    object
 6   OCCUPATION      247 non-null    object
 7   PERSON          231 non-null    object
 8   REGION          348 non-null    object
 9   ORG             541 non-null    object
 10  STREET          618 non-null    object
 11  MISC            27 non-null     object
 12  LOC             49 non-null     object
 13  DATE            11 non-null     object
dtypes: int64(3), object(11)
memory usage: 67.7+ KB


In [19]:
drop = ['ok', 'quality', 'status', 'bbox', 'confidence', 'place']

df_base = pd.read_excel('enderecos.xlsx', sheet_name=0, index_col=0)
df_base.dropna(subset=['lat', 'lng'], axis=0, inplace=True)
df_base.drop(columns=drop, inplace=True)
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, Rua William Vilas Boas to Rua Diego Donha de Proença
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   address  238 non-null    object 
 1   city     203 non-null    object 
 2   country  206 non-null    object 
 3   lat      239 non-null    float64
 4   lng      239 non-null    float64
 5   state    205 non-null    object 
 6   postal   95 non-null     object 
dtypes: float64(2), object(5)
memory usage: 14.9+ KB


In [20]:
df_base.head()

,address,city,country,lat,lng,state,postal
Rua William Vilas Boas,Rua William Vilas Boas Santo Antônio da Platin...,Santo Antônio da Platina,Brazil,-23.301637,-50.065405,Paraná,NaN
Rua Sete de Setembro,Rua Sete De Setembro Santo Antônio da Platina ...,Santo Antônio da Platina,Brazil,-23.294075,-50.077939,Paraná,86430-000
Rua Major Infante,Rua Major Infante Santo Antônio da Platina - P...,Santo Antônio da Platina,Brazil,-23.294611,-50.068227,Paraná,86430-000
Rua Marechal Deodoro da Fonseca,Rua Marechal Deodoro Da Fonseca Santo Antônio ...,Santo Antônio da Platina,Brazil,-23.291620,-50.071342,Paraná,86430-000
Rua Antônio Zanete,Antônio Zanete Santo Antônio da Platina - Para...,Santo Antônio da Platina,Brazil,-23.294931,-50.055520,Paraná,NaN


In [21]:
df = df_street.merge(df_base, left_on='STREET', right_index=True)

In [22]:
df.head()

,ano,numero,ementa,autores,id,texto_original,OCCUPATION,PERSON,REGION,ORG,...,MISC,LOC,DATE,address,city,country,lat,lng,state,postal
0,2021,5,Solicita ao Executivo pavimentação asfáltica n...,14,2671,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,"('Jardim Monte das Oliveiras', 'Bairro Santo A...","('Executivo',)",...,NaN,NaN,NaN,Rua William Vilas Boas Santo Antônio da Platin...,Santo Antônio da Platina,Brazil,-23.301637,-50.065405,Paraná,NaN
1,2021,7,Solicita ao Executivo a implantação de faixa e...,14,2673,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,"('Vila Santa Terezinha',)","('Executivo',)",...,NaN,NaN,NaN,Rua William Vilas Boas Santo Antônio da Platin...,Santo Antônio da Platina,Brazil,-23.301637,-50.065405,Paraná,NaN
395,2022,290,Solicita à Secretaria Municipal de Serviços e ...,14,3781,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,"('Jardim Bela Manhã', 'Santo Antônio da Platin...",('Secretaria Municipal de Serviços e Obras Púb...,...,NaN,NaN,NaN,Rua William Vilas Boas Santo Antônio da Platin...,Santo Antônio da Platina,Brazil,-23.301637,-50.065405,Paraná,NaN
402,2022,318,Solicita à Secretaria Municipal de Serviços e ...,142,3819,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,"('Jardim Bela Manhã', 'Santo Antônio da Platin...",('Secretaria Municipal de Serviços e Obras Púb...,...,NaN,NaN,NaN,Rua William Vilas Boas Santo Antônio da Platin...,Santo Antônio da Platina,Brazil,-23.301637,-50.065405,Paraná,NaN
2,2021,14,Solicita ao Executivo uma faixa elevada na Rua...,20,2683,https://sapl.santoantoniodaplatina.pr.leg.br/m...,NaN,NaN,NaN,"('Executivo',)",...,NaN,NaN,NaN,Rua Treze De Maio Santo Antônio da Platina - P...,Santo Antônio da Platina,Brazil,-23.297476,-50.076383,Paraná,86430-000


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536 entries, 0 to 616
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ano             536 non-null    int64  
 1   numero          536 non-null    int64  
 2   ementa          536 non-null    object 
 3   autores         536 non-null    object 
 4   id              536 non-null    int64  
 5   texto_original  536 non-null    object 
 6   OCCUPATION      214 non-null    object 
 7   PERSON          204 non-null    object 
 8   REGION          286 non-null    object 
 9   ORG             468 non-null    object 
 10  STREET          536 non-null    object 
 11  MISC            22 non-null     object 
 12  LOC             43 non-null     object 
 13  DATE            10 non-null     object 
 14  address         535 non-null    object 
 15  city            453 non-null    object 
 16  country         464 non-null    object 
 17  lat             536 non-null    float64


In [23]:
len(df)

536

In [25]:
df.to_csv('requerimentos-ruas-3.csv', index=False)